In [2]:
# for sample in samples:
#     sample["conditions"] = sample["conditions"][0]
#     sample["procedures"] = sample["procedures"][0]
#     sample["drugs"] = sample["drugs"][0]

In [9]:
TASK = "mortality"
DATASET = "mimic3"


In [10]:
from pyhealth.datasets import SampleEHRDataset
import json

with open(f"/shared/eng/pj20/kelpie_exp_data/ehr_data/{DATASET}_{TASK}_samples_train.json", "r") as f:
    samples_train = json.load(f)
with open(f"/shared/eng/pj20/kelpie_exp_data/ehr_data/{DATASET}_{TASK}_samples_test.json", "r") as f:
    samples_test = json.load(f)


dataset_train = SampleEHRDataset(samples_train, dataset_name=DATASET, task_name=TASK)
dataset_test = SampleEHRDataset(samples_test, dataset_name=DATASET, task_name=TASK)


In [ ]:
len(dataset_test)

In [12]:
from pyhealth.datasets import split_by_patient, get_dataloader

# train_dataset, val_dataset, test_dataset = split_by_patient(
#     dataset, [0.8, 0.1, 0.1], seed=528
# )
train_dataloader = get_dataloader(dataset_train, batch_size=32, shuffle=True)
test_dataloader = get_dataloader(dataset_test, batch_size=32, shuffle=False)

In [18]:
from pyhealth.trainer import Trainer
from pyhealth.models import Deepr, AdaCare, StageNet, GRASP, Transformer, RETAIN, RNN
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

model = GRASP(
    dataset=dataset_train,
    feature_keys=["conditions", "procedures", "drugs"],
    label_key="label",
    mode="binary",
    use_embedding=[True, True, True],
    embedding_dim=128,
)


In [ ]:
trainer = Trainer(model=model, metrics=['accuracy', 'f1', 'pr_auc', 'sensitivity', 'specificity'])
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=test_dataloader,
    epochs=15,
    optimizer_params = {"lr": 1e-3},
    monitor="f1",
)

# STEP 5: evaluate
print(trainer.evaluate(test_dataloader))